In [1]:
import numpy as np
import tensorflow.keras
from tensorflow.keras import backend as K
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.models import load_model

import itertools
import matplotlib.pyplot as plt
from datetime import datetime
%matplotlib inline
import os

In [2]:
train_path = 'fruits-360/Training'
test_path = 'fruits-360/Test'
classes = ['Apple Braeburn', 'Apple Crimson Snow', 'Apple Golden 1', 'Apple Golden 2', 'Apple Golden 3', 'Apple Granny Smith', 'Apple Pink Lady', 'Apple Red 1', 'Apple Red 2', 'Apple Red 3', 'Apple Red Delicious', 'Apple Red Yellow 1', 'Apple Red Yellow 2', 'Apricot', 'Avocado', 'Avocado ripe', 'Banana', 'Banana Lady Finger', 'Banana Red', 'Beetroot', 'Blueberry', 'Cactus fruit', 'Cantaloupe 1', 'Cantaloupe 2', 'Carambula', 'Cauliflower', 'Cherry 1', 'Cherry 2', 'Cherry Rainier', 'Cherry Wax Black', 'Cherry Wax Red', 'Cherry Wax Yellow', 'Chestnut', 'Clementine', 'Cocos', 'Corn', 'Corn Husk', 'Cucumber Ripe', 'Cucumber Ripe 2', 'Dates', 'Eggplant', 'Fig', 'Ginger Root', 'Granadilla', 'Grape Blue', 'Grape Pink', 'Grape White', 'Grape White 2', 'Grape White 3', 'Grape White 4', 'Grapefruit Pink', 'Grapefruit White', 'Guava', 'Hazelnut', 'Huckleberry', 'Kaki', 'Kiwi', 'Kohlrabi', 'Kumquats', 'Lemon', 'Lemon Meyer', 'Limes', 'Lychee', 'Mandarine', 'Mango', 'Mango Red', 'Mangostan', 'Maracuja', 'Melon Piel de Sapo', 'Mulberry', 'Nectarine', 'Nectarine Flat', 'Nut Forest', 'Nut Pecan', 'Onion Red', 'Onion Red Peeled', 'Onion White', 'Orange', 'Papaya', 'Passion Fruit', 'Peach', 'Peach 2', 'Peach Flat', 'Pear', 'Pear 2', 'Pear Abate', 'Pear Forelle', 'Pear Kaiser', 'Pear Monster', 'Pear Red', 'Pear Stone', 'Pear Williams', 'Pepino', 'Pepper Green', 'Pepper Orange', 'Pepper Red', 'Pepper Yellow', 'Physalis', 'Physalis with Husk', 'Pineapple', 'Pineapple Mini', 'Pitahaya Red', 'Plum', 'Plum 2', 'Plum 3', 'Pomegranate', 'Pomelo Sweetie', 'Potato Red', 'Potato Red Washed', 'Potato Sweet', 'Potato White', 'Quince', 'Rambutan', 'Raspberry', 'Redcurrant', 'Salak', 'Strawberry', 'Strawberry Wedge', 'Tamarillo', 'Tangelo', 'Tomato 1', 'Tomato 2', 'Tomato 3', 'Tomato 4', 'Tomato Cherry Red', 'Tomato Heart', 'Tomato Maroon', 'Tomato not Ripened', 'Tomato Yellow', 'Walnut', 'Watermelon']

In [3]:
#Data augmentation and validation split
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    validation_split=0.2) # set validation split

batch_size=50

train_batches = train_datagen.flow_from_directory(train_path, target_size=(100,100), classes=classes, batch_size=batch_size, subset='training')
validation_batches = train_datagen.flow_from_directory(train_path, target_size=(100,100), classes=classes, batch_size=batch_size, subset='validation')
test_batches = ImageDataGenerator(rescale=1./255).flow_from_directory(test_path, target_size=(100,100), classes=classes, batch_size=batch_size)

Found 54190 images belonging to 131 classes.
Found 13502 images belonging to 131 classes.
Found 22688 images belonging to 131 classes.


# Build and train CNN

In [4]:
dropout_rate = 0.5

model = Sequential([
    Conv2D(16, (3,3), activation='relu', input_shape=(100,100,3), padding='valid'),#100x100
    MaxPooling2D(pool_size=(2, 2), strides=2),#98x98x16
    Conv2D(32, (3,3), activation='relu', padding='valid'),#49x49x16
    MaxPooling2D(pool_size=(2, 2), strides=2),#47x47x32
    Conv2D(64, (3,3), activation='relu', padding='valid'),#23x23x32
    MaxPooling2D(pool_size=(2, 2), strides=2),#21x21x64
    Conv2D(128, (3,3), activation='relu', padding='valid'),#10x10x64
    MaxPooling2D(pool_size=(2, 2), strides=2),#8x8x128
    Conv2D(256, (3,3), activation='relu', padding='valid'),#4x4x128
    Flatten(),#4x4x256
    Dense(2048, activation='relu'),#4096
    Dropout(dropout_rate),
    Dense(512, activation='relu'),
    Dropout(dropout_rate),
    Dense(131, activation='softmax'),
])

In [5]:
learning_rate=1e-4
loss_function='categorical_crossentropy'
model.compile(Adam(lr=learning_rate), loss=loss_function, metrics=['accuracy'])

In [6]:
# datetime object containing current date and time
now = datetime.now()
# dd/mm/YY H:M:S
dt_string = now.strftime("%d-%m-%Y_%H-%M-%S")
iter_count=0

In [7]:
if not os.path.exists("models/scratch"+dt_string):
    os.mkdir("models/scratch/"+dt_string)
    
summary_file = open("models/scratch/"+dt_string+"/summary.txt", "w")

def printModelSummaryToFile(line):
    summary_file.write(line+"\n")

In [ ]:
model.summary(print_fn=printModelSummaryToFile)
summary_file.write("\nLearning Rate: "+str(learning_rate)+"\n")
summary_file.write("Loss Function: "+str(loss_function)+"\n")
summary_file.write("Dropout Rate: "+str(dropout_rate)+"\n")
summary_file.write("Batch Size: "+str(batch_size)+"\n")
summary_file.close()

#Callbacks
es = EarlyStopping(monitor='val_accuracy', mode='max', verbose=1, patience=25)
mc = ModelCheckpoint("models/scratch/"+dt_string+"/"+dt_string+"_best.h5", monitor='val_accuracy', mode='max', save_best_only=True)

start_time = datetime.now()

history = model.fit(x=train_batches, validation_data=validation_batches, epochs=999, verbose=2, callbacks=[es, mc])

end_time = datetime.now()
time_delta =(end_time-start_time).total_seconds()

summary_file = open("models/scratch/"+dt_string+"/summary.txt", "a")
summary_file.write("Time Train Start: "+str(start_time)+"\n")
summary_file.write("Time Train End: "+str(end_time)+"\n")
summary_file.write("Time Delta: "+str(time_delta)+"\n")
summary_file.close()

model.save("models/scratch/"+dt_string+"/"+dt_string+"_last.h5")

Epoch 1/999
1084/1084 - 157s - loss: 2.9892 - accuracy: 0.2497 - val_loss: 1.2021 - val_accuracy: 0.6894
Epoch 2/999
1084/1084 - 135s - loss: 1.0298 - accuracy: 0.6774 - val_loss: 0.5758 - val_accuracy: 0.8441
Epoch 3/999
1084/1084 - 138s - loss: 0.5162 - accuracy: 0.8342 - val_loss: 0.4571 - val_accuracy: 0.8779
Epoch 4/999
1084/1084 - 138s - loss: 0.3041 - accuracy: 0.9006 - val_loss: 0.3313 - val_accuracy: 0.9122
Epoch 5/999
1084/1084 - 135s - loss: 0.2123 - accuracy: 0.9311 - val_loss: 0.3202 - val_accuracy: 0.9252
Epoch 6/999
1084/1084 - 134s - loss: 0.1547 - accuracy: 0.9482 - val_loss: 0.2911 - val_accuracy: 0.9350
Epoch 7/999
1084/1084 - 134s - loss: 0.1187 - accuracy: 0.9618 - val_loss: 0.2875 - val_accuracy: 0.9382
Epoch 8/999
1084/1084 - 133s - loss: 0.0980 - accuracy: 0.9685 - val_loss: 0.3085 - val_accuracy: 0.9351
Epoch 9/999
1084/1084 - 132s - loss: 0.0777 - accuracy: 0.9737 - val_loss: 0.3698 - val_accuracy: 0.9229
Epoch 10/999
1084/1084 - 134s - loss: 0.0733 - accuracy

# Prediction

In [ ]:
best_model = load_model("models/scratch/"+dt_string+"/"+dt_string+"_best.h5")

In [ ]:
test_imgs, test_labels = next(test_batches)

In [ ]:
test_labels = np.argmax(test_labels, axis=-1)
test_labels

In [ ]:
predictions = np.argmax(best_model.predict(x=test_imgs, steps=1, verbose=0), axis=-1)

In [ ]:
predictions

In [ ]:
# plot training history
plt.title('Loss')
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='test')
plt.legend()
plt.savefig("models/scratch/"+dt_string+"/loss_history.png", bbox_inches='tight')
plt.show()

plt.title('Accuracy')
plt.plot(history.history['accuracy'], label='train')
plt.plot(history.history['val_accuracy'], label='test')
plt.legend()
plt.savefig("models/scratch/"+dt_string+"/accuracy_history.png", bbox_inches='tight')
plt.show()

In [ ]:
# evaluate the model
_, train_acc = best_model.evaluate(train_batches, verbose=1)
_, test_acc = best_model.evaluate(test_batches, verbose=1)
print('Train: %.3f, Test: %.3f' % (train_acc, test_acc))
summary_file = open("models/scratch/"+dt_string+"/summary.txt", "a")
summary_file.write("Train Accuracy: "+str(train_acc)+"\n")
summary_file.write("Test Accuracy: "+str(test_acc)+"\n")
summary_file.close()

In [ ]:
best_model.summary()